In [8]:
import os
import sys

sys.path.append(os.getcwd() + "/spider")

print(sys.path)

['/Users/lanch/miniconda3/envs/handsonllm/lib/python311.zip', '/Users/lanch/miniconda3/envs/handsonllm/lib/python3.11', '/Users/lanch/miniconda3/envs/handsonllm/lib/python3.11/lib-dynload', '', '/Users/lanch/miniconda3/envs/handsonllm/lib/python3.11/site-packages', '/Users/lanch/Desktop/Code/tcb/handsonllm/spider_test/spider', '/Users/lanch/Desktop/Code/tcb/handsonllm/spider_test/spider', '/Users/lanch/Desktop/Code/tcb/handsonllm/spider_test/spider']


In [9]:
from spider.evaluation import evaluate, build_foreign_key_map_from_json
import pandas as pd


def run_evaluate(model: str, dataset: str):
    result = evaluate(
        gold=f"spider_data/{dataset}_gold.sql",
        predict=f"inference_data/{model}_{dataset}_inf.txt",
        db_dir=(
            "spider_data/database" if dataset == "dev" else "spider_data/test_database"
        ),
        etype="exec",
        kmaps=build_foreign_key_map_from_json(
            "spider_data/tables.json"
            if dataset == "dev"
            else "spider_data/test_tables.json"
        ),
    )
    return {
        "model": model,
        "dataset": dataset,
        "easy": f"{result['easy']['exec'] * 100:.2f}% over {result['easy']['count']}",
        "medium": f"{result['medium']['exec'] * 100:.2f}% over {result['medium']['count']}",
        "hard": f"{result['hard']['exec'] * 100:.2f}% over {result['hard']['count']}",
        "extra": f"{result['extra']['exec'] * 100:.2f}% over {result['extra']['count']}",
        "all": f"{result['all']['exec'] * 100:.2f}% over {result['all']['count']}",
    }

In [13]:
eval_list = [
    {"model": "llama32_3b", "dataset": "dev"},
    {"model": "llama32_3b", "dataset": "test"},
    {"model": "xiyansql_7b_8_cn_prompt", "dataset": "dev"},
    {"model": "xiyansql_7b_8_en_prompt", "dataset": "dev"},
    {"model": "xiyansql_7b_8_en_prompt", "dataset": "test"},
    {"model": "xiyansql_7b_16_cn_prompt", "dataset": "dev"},
    {"model": "xiyansql_7b_16_en_prompt", "dataset": "dev"},
    {"model": "xiyansql_7b_16_en_prompt_vllm", "dataset": "dev"},
    {"model": "xiyansql_7b_16_en_prompt_vllm", "dataset": "test"},
]

data = []
for config in eval_list:
    res = run_evaluate(**config)
    data.append(res)

eval_err_num:1
AssertionError('Error col: country=__val_62_69__')
eval_err_num:2
AssertionError('Error col: nationality')
eval_err_num:3
KeyError('inner')
eval_err_num:4
KeyError('inner')
eval_err_num:5
KeyError('inner')
eval_err_num:6
KeyError('inner')
eval_err_num:7
KeyError('country')
eval_err_num:8
KeyError('inner')
eval_err_num:9
KeyError('inner')
eval_err_num:10
KeyError('stations')
eval_err_num:11
AssertionError()
eval_err_num:12
KeyError('stadiums')
eval_err_num:13
AssertionError('Error col: avg_attendance')
eval_err_num:14
AssertionError('Error col: avg_attendance')
eval_err_num:15
AssertionError()
eval_err_num:16
AssertionError()
eval_err_num:17
KeyError('concerts')
eval_err_num:18
AssertionError('Error col: stadium')
eval_err_num:19
KeyError('concerts')
eval_err_num:20
KeyError('stadium.id')
eval_err_num:21
AssertionError()
eval_err_num:22
KeyError('concert_id')
eval_err_num:23
AssertionError('Error col: stadium')
eval_err_num:24
KeyError('concerts')
eval_err_num:25
KeyError

In [14]:
df = pd.DataFrame(data)
df

,model,dataset,easy,medium,hard,extra,all
0,llama32_3b,dev,24.19% over 248,5.83% over 446,5.75% over 174,1.20% over 166,9.48% over 1034
1,llama32_3b,test,20.85% over 470,8.40% over 857,4.32% over 463,1.68% over 357,9.13% over 2147
2,xiyansql_7b_8_cn_prompt,dev,83.06% over 248,72.20% over 446,61.49% over 174,51.20% over 166,69.63% over 1034
3,xiyansql_7b_8_en_prompt,dev,81.85% over 248,74.22% over 446,59.77% over 174,50.00% over 166,69.73% over 1034
4,xiyansql_7b_8_en_prompt,test,81.49% over 470,70.83% over 857,58.96% over 463,51.54% over 357,67.40% over 2147
5,xiyansql_7b_16_cn_prompt,dev,83.47% over 248,73.09% over 446,61.49% over 174,51.81% over 166,70.21% over 1034
6,xiyansql_7b_16_en_prompt,dev,82.66% over 248,73.99% over 446,60.34% over 174,49.40% over 166,69.83% over 1034
7,xiyansql_7b_16_en_prompt_vllm,dev,82.26% over 248,72.65% over 446,55.75% over 174,53.01% over 166,68.96% over 1034
8,xiyansql_7b_16_en_prompt_vllm,test,80.43% over 470,72.23% over 857,59.40% over 463,54.06% over 357,68.23% over 2147
